<a href="https://colab.research.google.com/github/Tahira2910/ETG_Python_Projects/blob/main/2_Cryptocurrency_Price_Prediction_using_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install autots

In [2]:
import pandas as pd

import yfinance as yf    # collecting the latest Bitcoin prices data from Yahoo Finance, using the yfinance API


import datetime          # It allows you to create, manipulate, format, and perform calculations on dates and times


# The date class represents a date (year, month, day) without time, while the timedelta class represents a duration or difference between two dates or times
from datetime import date, timedelta 

# This line creates a variable named "today" and assigns it the value of 
# the current date. The date.today() function returns the current local date based on the system's clock. 
today = date.today()

In [3]:
d1 = today.strftime("%Y-%m-%d")  # converts the current date (today) into a string format with the specified format "%Y-%m-%d"
end_date = d1                    # assigns the value of the formatted date string (d1) to the variable end_date
d2 = date.today() - timedelta(days=730)  #calculates a new date by subtracting 730 days (two years) from the current date (date.today())
d2 = d2.strftime("%Y-%m-%d")
start_date = d2

data = yf.download('BTC-USD',   #BTC-USD represents the stock symbol for Bitcoin,  'USD' indicates that the value of Bitcoin is being measured in U.S. dollars.
                      start=start_date,  #specifies the start date and end date for retrieving the financial data
                      end=end_date, 
                      progress=False)  #disable the progress bar display during the download.
data["Date"] = data.index
data = data[["Date", "Open", "High", "Low", "Close", "Adj Close", "Volume"]] #selects specific columnsfrom the DataFrame data and assigns the resulting DataFrame back to data
data.reset_index(drop=True, inplace=True) #drop=True--> parameter is used to discard the previous index, inplace=True--> modifying the original DataFrame.
print(data.head())


        Date          Open          High           Low         Close  \
0 2021-06-13  35555.789062  39322.781250  34864.109375  39097.859375   
1 2021-06-14  39016.968750  40978.363281  38757.285156  40218.476562   
2 2021-06-15  40427.167969  41295.269531  39609.468750  40406.269531   
3 2021-06-16  40168.691406  40516.777344  38176.035156  38347.062500   
4 2021-06-17  38341.421875  39513.671875  37439.675781  38053.503906   

      Adj Close       Volume  
0  39097.859375  40669112838  
1  40218.476562  43148914673  
2  40406.269531  46420149185  
3  38347.062500  39211635100  
4  38053.503906  37096670047  


In [4]:
data.shape #shows number of rows and the number of columns in the DataFrame.

(730, 7)

In [5]:
import plotly.graph_objects as go # used this module to create a candlestick chart 
# creates new figure object named figure  using the go.Figure() constructor
# The data parameter is specified as a list containing a single element, which is the go.Candlestick() object
figure = go.Figure(data=[go.Candlestick(x=data["Date"],
                                        open=data["Open"],  # represents the opening prices of the candlestick chart
                                        high=data["High"], #  represents the high prices of the candlestick char
                                        low=data["Low"],    # represents the low prices of the candlestick chart
                                        close=data["Close"])])  #  represents the closing prices of the candlestick chart
figure.update_layout(title = "Bitcoin Price Analysis", 
                     xaxis_rangeslider_visible=False) #  hides the range slider for the x-axis.
figure.show()

In [6]:
correlation = data.corr() # calculates the correlation matrix between all columns in the DataFrame data using the corr() method. 

# prints the correlation values for the "Close" column. It sorts these 
# correlation values in descending order using the sort_values() method with the ascending=False parameter. 
print(correlation["Close"].sort_values(ascending=False)) 

Close        1.000000
Adj Close    1.000000
High         0.998161
Low          0.998075
Open         0.995792
Volume       0.193262
Name: Close, dtype: float64


<ipython-input-6-81091e368782>:1: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



In [7]:
from autots import AutoTS
# frequency='infer-->  implies that the frequency will be automatically inferred from the data.
# method to use for combining multiple forecasting models. Here, the 'simple' ensemble method is chosen.
# The reason for choosing the "simple" ensemble method can vary based on factors such as the dataset,
#  the desired level of complexity, and the available computational resources. 
model = AutoTS(forecast_length=30, frequency='infer', ensemble='simple')
model = model.fit(data, date_col='Date', value_col='Close', id_col=None)
prediction = model.predict()
forecast = prediction.forecast
print(forecast)

Inferred frequency is: D
Model Number: 1 with model ARIMA in generation 0 of 10
Model Number: 2 with model AverageValueNaive in generation 0 of 10
Model Number: 3 with model AverageValueNaive in generation 0 of 10
Model Number: 4 with model AverageValueNaive in generation 0 of 10
Model Number: 5 with model DatepartRegression in generation 0 of 10
Model Number: 6 with model DatepartRegression in generation 0 of 10
Model Number: 7 with model DatepartRegression in generation 0 of 10


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html



Model Number: 8 with model DatepartRegression in generation 0 of 10
Epoch 1/50
22/22 [==============================] - 20s 12ms/step - loss: 0.4171
Epoch 2/50
22/22 [==============================] - 0s 12ms/step - loss: 0.4159
Epoch 3/50
22/22 [==============================] - 0s 10ms/step - loss: 0.4154
Epoch 4/50
22/22 [==============================] - 0s 11ms/step - loss: 0.4150
Epoch 5/50
22/22 [==============================] - 0s 12ms/step - loss: 0.4157
Epoch 6/50
22/22 [==============================] - 0s 14ms/step - loss: 0.4137
Epoch 7/50
22/22 [==============================] - 0s 15ms/step - loss: 0.4145
Epoch 8/50
22/22 [==============================] - 0s 14ms/step - loss: 0.4148
Epoch 9/50
22/22 [==============================] - 0s 10ms/step - loss: 0.4141
Epoch 10/50
22/22 [==============================] - 0s 14ms/step - loss: 0.4166
Epoch 11/50
22/22 [==============================] - 0s 14ms/step - loss: 0.4144
Epoch 12/50
22/22 [==============================

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html



Model Number: 36 with model FBProphet in generation 0 of 10


...

183 - GLS with avg smape 5.46: 
Model Number: 184 of 191 with model DatepartRegression for Validation 3
184 - DatepartRegression with avg smape 10.69: 
Model Number: 185 of 191 with model DatepartRegression for Validation 3
185 - DatepartRegression with avg smape 52.69: 
Model Number: 186 of 191 with model DatepartRegression for Validation 3


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.



186 - DatepartRegression with avg smape 200.0: 
Model Number: 187 of 191 with model DatepartRegression for Validation 3
📈 187 - DatepartRegression with avg smape 4.3: 
Model Number: 188 of 191 with model DatepartRegression for Validation 3
188 - DatepartRegression with avg smape 5.71: 
Model Number: 189 of 191 with model GLM for Validation 3
189 - GLM with avg smape 27.04: 
Model Number: 190 of 191 with model DatepartRegression for Validation 3


/usr/local/lib/python3.10/dist-packages/statsmodels/genmod/generalized_linear_model.py:298: DomainWarning:

The inverse_power link function does not respect the domain of the Gamma family.



190 - DatepartRegression with avg smape 18.84: 
Model Number: 191 of 191 with model GLM for Validation 3
191 - GLM with avg smape 38.41: 
Model Number: 1843 with model Ensemble in generation 12 of Ensembles
Model Number: 1844 with model Ensemble in generation 12 of Ensembles
Model Number: 1845 with model Ensemble in generation 12 of Ensembles
Model Number: 1846 with model Ensemble in generation 12 of Ensembles
Model Number: 1847 with model Ensemble in generation 12 of Ensembles
Model Number: 1848 with model Ensemble in generation 12 of Ensembles
Model Number: 1849 with model Ensemble in generation 12 of Ensembles
Model Number: 1850 with model Ensemble in generation 12 of Ensembles
Validation Round: 1
Model Number: 1 of 8 with model Ensemble for Validation 1
📈 1 - Ensemble with avg smape 2.92: 
Model Number: 2 of 8 with model Ensemble for Validation 1
2 - Ensemble with avg smape 2.94: 
Model Number: 3 of 8 with model Ensemble for Validation 1
3 - Ensemble with avg smape 4.32: 
Model Num

 where the forecasts from multiple models are averaged or combined using equal weights

 **ensemble=mean**--> calculates the mean forecast across multiple models
** ensemble='median-->** calculates the median forecast across multiple models

**model = model.fit(data, date_col='Date', value_col='Close', id_col='Ticker')**

By specifying id_col='Ticker', the fit() method will consider each unique value in the "Ticker" column as a separate time series. The model will then fit and forecast each time series individually, capturing the specific patterns and behavior of each stock. when the model is fitted with id_col specified, it will train and learn from each time series separately, enabling it to generate forecasts that account

This is useful when you want to analyze and forecast different time series separately, accounting for the unique characteristics and trends of each series. The id_col parameter allows you to handle such multi-series datasets and perform separate modeling and forecasting as needed.

The Close column in the dataset contains the values we need to predict. So, let’s have a look at the correlation of all the columns in the data concerning the Close column: